In [53]:
import xml.etree.ElementTree as ET
import pandas as pd
import os
pd.set_option('display.max_columns', 500)

In [37]:
files_to_parse = [ file for file in os.listdir() if '.xml' in file ]

In [22]:
with open('gid_2019_03_28_houmlb_tbamlb_1.xml', 'r') as file:
    root = ET.parse(file).getroot()

In [57]:
at_bats = []
for file in files_to_parse:
    with open(file, 'r') as file:
        root = ET.parse(file).getroot()
    for atbat in root.iter(tag='atbat'):
        for pitch in atbat.iter(tag='pitch'):
            data = pitch.attrib
            data["batter"] = atbat.attrib["batter"]
            data["pitcher"] = atbat.attrib["pitcher"]
            data["atbat_result"] = atbat.attrib["event"]
            at_bats.append(data)

In [58]:
df_ab = pd.DataFrame(at_bats)

In [59]:
df_ab.shape

(32870, 43)

In [60]:
df_ab.head()

,atbat_result,ax,ay,az,batter,break_angle,break_length,break_y,cc,code,des,des_es,end_speed,event_num,id,mt,nasty,pfx_x,pfx_z,pitch_type,pitcher,play_guid,px,pz,spin_dir,spin_rate,start_speed,sv_id,sz_bot,sz_top,tfs,tfs_zulu,type,type_confidence,vx0,vy0,vz0,x,x0,y,y0,z0,zone
0,Flyout,-18.14,25.67,-25.86,640457,32.4,7.2,24.0,,B,Ball,"In play, out(s)",83.8,2,2,,,-10.15,3.53,FT,519144,beda73fc-8d1d-4303-9ce4-579c11b31816,-0.93,3.47,placeholder,placeholder,90.9,190607_231050,1.58,3.42,231045,2019-06-07T23:10:45.000Z,B,placeholder,6.71,-132.22,-1.66,152.29,-2.16,145.27,50.0,5.98,placeholder
1,Flyout,-11.49,25.24,-17.76,640457,28.8,4.8,24.0,,C,Called Strike,Ball,84.1,3,3,,,-6.39,8.03,FF,519144,25368d67-865c-493d-a494-7d38c66372df,-0.64,2.79,placeholder,placeholder,91.1,190607_231104,1.66,3.48,231058,2019-06-07T23:10:58.000Z,S,placeholder,5.94,-132.43,-4.78,141.57,-2.07,163.43,50.0,5.9,placeholder
2,Flyout,-11.34,27.9,-16.18,640457,31.2,4.8,24.0,,B,Ball,Called Strike,83.4,4,4,,,-6.35,8.95,FF,519144,079bcb5f-a5c6-408d-b96f-bbee5b2d1c35,-0.25,4.18,placeholder,placeholder,91.2,190607_231118,1.61,3.47,231113,2019-06-07T23:11:13.000Z,B,placeholder,5.93,-132.68,-2.23,126.69,-1.69,125.97,50.0,6.21,placeholder
3,Flyout,-18.08,28.25,-27.16,640457,30.0,7.2,24.0,,X,"In play, out(s)",Ball,82.8,5,5,,,-10.34,2.87,FT,519144,2aab16a9-5ee2-4b16-9496-a1973b97d621,0.31,2.35,placeholder,placeholder,90.6,190607_231144,1.58,3.41,231129,2019-06-07T23:11:29.000Z,X,placeholder,9.48,-131.47,-3.48,105.16,-2.0,175.36,50.0,5.71,placeholder
4,Groundout,-15.04,25.55,-28.16,502054,26.4,7.2,24.0,,C,Called Strike,"In play, out(s)",84.1,8,8,,,-8.38,2.24,FT,519144,378a3338-e56c-41d7-a274-81ac6dd2958e,-0.81,3.45,placeholder,placeholder,91.0,190607_231209,1.59,3.33,231208,2019-06-07T23:12:08.000Z,S,placeholder,6.2,-132.47,-1.04,147.85,-2.08,145.75,50.0,5.88,placeholder


In [61]:
meadows_id = 640457

In [62]:
meadows_df = df_ab.query("batter == '{}'".format(meadows_id))

In [75]:
from functools import partial

to_datetime_format = partial(pd.to_datetime, format="%Y-%m-%dT%H:%M:%S.%fZ")

In [77]:
time_index = meadows_df.tfs_zulu.apply(to_datetime_format)

In [88]:
meadows_df.tail()

,atbat_result,ax,ay,az,batter,break_angle,break_length,break_y,cc,code,des,des_es,end_speed,event_num,id,mt,nasty,pfx_x,pfx_z,pitch_type,pitcher,play_guid,px,pz,spin_dir,spin_rate,start_speed,sv_id,sz_bot,sz_top,tfs,tfs_zulu,type,type_confidence,vx0,vy0,vz0,x,x0,y,y0,z0,zone
32840,Single,-2.90,26.47,-13.29,640457,10.8,3.6,24.0,NaN,F,Foul,Ball,85.6,420,420,NaN,NaN,-1.56,10.15,FF,453284,e39fbac6-2bfa-4ef8-9acf-b91f6d407c57,-0.28,2.24,placeholder,placeholder,92.6,190627_024731,1.59,3.39,24726,2019-06-27T02:47:26.000Z,S,placeholder,0.93,-134.84,-7.80,127.75,-0.43,178.45,50.0,6.08,placeholder
32841,Single,-6.17,19.15,-31.55,640457,10.8,9.6,24.0,NaN,F,Foul,Foul,75.0,421,421,NaN,NaN,-4.38,0.45,FS,453284,8556ea6f-07c9-491d-87aa-dcafce24859d,-0.09,2.23,placeholder,placeholder,80.3,190627_024806,1.59,3.39,24800,2019-06-27T02:48:00.000Z,S,placeholder,2.20,-117.09,-2.39,120.34,-0.46,178.55,50.0,6.17,placeholder
32842,Single,-3.95,23.74,-17.19,640457,10.8,3.6,24.0,NaN,F,Foul,Foul,86.5,422,422,NaN,NaN,-2.10,7.98,FF,453284,b777f56c-af52-4620-9b0d-7e83cba22c95,0.75,1.55,placeholder,placeholder,92.6,190627_024856,1.59,3.39,24850,2019-06-27T02:48:50.000Z,S,placeholder,3.40,-134.82,-8.91,88.37,-0.24,196.91,50.0,6.06,placeholder
32843,Single,-3.37,26.24,-13.87,640457,9.6,3.6,24.0,NaN,F,Foul,Foul,85.4,423,423,NaN,NaN,-1.83,9.95,FF,453284,018f1159-f7af-49e4-ace7-5906f530b8c1,0.89,1.21,placeholder,placeholder,92.2,190627_024925,1.59,3.39,24920,2019-06-27T02:49:20.000Z,S,placeholder,3.94,-134.09,-10.10,83.16,-0.36,206.14,50.0,5.99,placeholder
32844,Single,-5.75,20.54,-32.30,640457,9.6,9.6,24.0,NaN,D,"In play, no out",Foul,74.2,424,424,NaN,NaN,-4.17,-0.09,FS,453284,99a0654f-a80f-4c42-9570-32693afaf8cb,-0.75,1.97,placeholder,placeholder,79.9,190627_025011,1.59,3.39,24958,2019-06-27T02:49:58.000Z,X,placeholder,0.87,-116.40,-2.59,145.48,-0.58,185.5,50.0,6.14,placeholder


In [89]:
# meadows_df.set_index(time_index).groupby([pd.Grouper(freq="W"),"pitch_type","atbat_result"]).size().unstack()

atbat_result           Caught Stealing 2B  Double  Double Play  Field Error  \
tfs_zulu   pitch_type                                                         
2019-03-31 CH                         NaN     NaN          NaN          NaN   
           CU                         NaN     NaN          NaN          NaN   
           FC                         NaN     NaN          NaN          NaN   
           FF                         NaN     NaN          NaN          2.0   
           FT                         NaN     NaN          NaN          NaN   
           KC                         NaN     NaN          NaN          1.0   
           SL                         NaN     NaN          NaN          NaN   
2019-04-07 CH                         NaN     2.0          NaN          NaN   
           CU                         NaN     NaN          NaN          NaN   
           FC                         NaN     1.0          NaN          NaN   
           FF                         NaN     1.0          NaN          NaN   
           FS                         NaN     NaN          NaN          NaN   
           FT                         NaN     NaN          NaN          NaN   
           KC                         NaN     NaN          NaN          NaN   
           SI                         NaN     NaN          NaN          NaN   
           SL                         NaN     NaN          NaN          NaN   
2019-04-14 CH                         NaN     1.0          NaN          1.0   
           CU                         NaN     NaN          NaN          NaN   
           FC                         NaN     1.0          NaN          NaN   
           FF                         NaN     5.0          NaN          1.0   
           FS                         NaN     NaN          NaN          NaN   
           FT                         NaN     NaN          NaN          NaN   
           SI                         NaN     NaN          NaN          NaN   
           SL                         NaN     1.0          NaN          1.0   
2019-04-21 CH                         NaN     1.0          2.0          NaN   
           CU                         NaN     NaN          NaN          NaN   
           FC                         NaN     NaN          NaN          NaN   
           FF                         2.0     1.0          3.0          NaN   
           FT                         NaN     1.0          NaN          NaN   
           KC                         2.0     NaN          NaN          NaN   
...                                   ...     ...          ...          ...   
2019-06-30 CH                         NaN     NaN          NaN          NaN   
           CU                         NaN     NaN          NaN          NaN   
           FC                         NaN     NaN          NaN          NaN   
           FF                         NaN     NaN          NaN          NaN   
           FS                         NaN     NaN          NaN          NaN   
           FT                         NaN     NaN          NaN          NaN   
           SI                         NaN     NaN          NaN          NaN   
           SL                         NaN     NaN          NaN          NaN   
2019-07-07 CH                         NaN     NaN          NaN          NaN   
           CU                         NaN     NaN          NaN          NaN   
           FC                         NaN     NaN          NaN          NaN   
           FF                         NaN     NaN          NaN          NaN   
           FS                         NaN     NaN          NaN          NaN   
           FT                         NaN     NaN          NaN          NaN   
           SI                         NaN     1.0          NaN          NaN   
           SL                         NaN     NaN          NaN          NaN   
2019-07-14 CH                         NaN     2.0          NaN          NaN   
           CU                         NaN     1.0 

In [87]:
to_numeric = partial(pd.to_numeric,errors="ignore")
meadows_df = meadows_df.apply(to_numeric)
meadows_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1431 entries, 0 to 32844
Data columns (total 43 columns):
atbat_result       1431 non-null object
ax                 1336 non-null float64
ay                 1336 non-null float64
az                 1336 non-null float64
batter             1431 non-null int64
break_angle        1336 non-null float64
break_length       1336 non-null float64
break_y            1336 non-null float64
cc                 0 non-null float64
code               1431 non-null object
des                1431 non-null object
des_es             1431 non-null object
end_speed          1336 non-null float64
event_num          1431 non-null int64
id                 1431 non-null int64
mt                 0 non-null float64
nasty              0 non-null float64
pfx_x              1336 non-null float64
pfx_z              1336 non-null float64
pitch_type         1336 non-null object
pitcher            1431 non-null int64
play_guid          1336 non-null object
px           

In [90]:
# meadows_df.to_pickle("meadows_thru_20190718.pkl")